In [1]:
import pandas as pd
import pymysql
import numpy as np

In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

# df = pd.read_sql_query("select * from master_data_view6A_master order by date", __connect__())

In [3]:
df = pd.read_sql_query("select * from g_master_data_joins_2 order by date", __connect__())

In [4]:
original_df = df.copy()

In [5]:
# df = original_df.copy()

In [6]:
df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,Transaction GMV,Interest GMV,Price/Sale,Users 24hr Debank,Transactions Count Debank,Trading Volume Debank,Borrowing Volume Debank,Users life-to-date duneanlytics,CoinUrl,Tags
0,2020-12-21,Amp,AMP,0.006596,6.361912e+07,369894.27,9.645497e+09,1.200105e+09,9.239914e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://amptoken.org,
1,2020-12-21,Ampleforth,AMPL,0.820038,1.885767e+08,10464139.25,2.299609e+08,NaN,4.102788e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://ampleforth.org,"defi,rebase,arrington-xrp-capital,alameda-rese..."
2,2020-12-21,Anyswap,ANY,0.198054,2.757083e+06,1703377.87,1.392088e+07,1.000000e+08,1.000000e+08,4.999804e+05,...,NaN,NaN,NaN,12.0,22.0,12187.952376,NaN,NaN,https://anyswap.exchange,"decentralized-exchange,defi,amm,binance-smart-..."
3,2020-12-21,APY.Finance,APY,0.232858,2.661404e+06,265178.54,1.142932e+07,NaN,1.000000e+08,9.500688e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://apy.finance,"defi,yield-farming,yield-aggregator,alameda-re..."
4,2020-12-21,Arweave,AR,2.628825,8.778883e+07,2873123.67,3.339470e+07,6.600000e+07,6.319044e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://arweave.org,"distributed-computing,filesharing,storage,coin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36302,2021-04-08,XVIX,XVIX,26.126145,0.000000e+00,4818.52,0.000000e+00,NaN,1.778200e+05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://xvix.finance,
36303,2021-04-08,DiFy.Finance,YFIII,252.282768,2.015815e+06,1032285.11,7.990300e+03,3.000000e+04,3.000000e+04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://dify.finance,"defi,yield-farming"
36304,2021-04-08,YIELD App,YLD,0.664795,6.252084e+07,2267896.50,9.404523e+07,3.000000e+08,1.068717e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://yield.app,
36305,2021-04-08,YVS.Finance,YVS,5.739416,2.173482e+06,239173.44,3.786940e+05,2.000000e+06,1.853611e+06,1.578759e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://yvs.finance,"defi,ethereum,yield-farming,binance-smart-chain"


In [7]:
original_df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank', 'TVL Defipulse', 'Transaction Revenue',
       'Interest Revenue', 'Transaction Revenue Supply Side',
       'Interest Revenue Supply Side', 'Transaction Revenue Protocol',
       'Interest Revenue Protocol', 'Transaction GMV', 'Interest GMV',
       'Price/Sale', 'Users 24hr Debank', 'Transactions Count Debank',
       'Trading Volume Debank', 'Borrowing Volume Debank',
       'Users life-to-date duneanlytics', 'CoinUrl', 'Tags'],
      dtype='object')

In [8]:
df['TVL_max'] = df[['TVL Debank', 'TVL Defipulse']].max(axis=1)

In [9]:
df[df['Symbol'] == 'BTC']['Interest Revenue']

132     NaN
187     NaN
496     NaN
551     NaN
865     NaN
         ..
34620   NaN
34689   NaN
35258   NaN
35840   NaN
35910   NaN
Name: Interest Revenue, Length: 109, dtype: float64

In [10]:
tran_gvm_df = df['Transaction GMV'].copy()
tran_gvm_df = tran_gvm_df.astype(object).where(pd.notnull(tran_gvm_df), 0)

int_gvm_df = df['Interest GMV'].copy()
int_gvm_df = int_gvm_df.astype(object).where(pd.notnull(int_gvm_df), 0)

df['GMV'] = (tran_gvm_df + int_gvm_df).replace(0, np.nan)

In [11]:
trading_volume_df = df['Trading Volume Debank'].copy()
trading_volume_df = trading_volume_df.astype(object).where(pd.notnull(trading_volume_df), 0)

borrowing_volume_df = df['Borrowing Volume Debank'].copy()
borrowing_volume_df = borrowing_volume_df.astype(object).where(pd.notnull(borrowing_volume_df), 0)

# df['Dex Volume DeBank'] = (trading_volume_df + borrowing_volume_df).replace(0, np.nan)
df['Defi Volume DeBank'] = (trading_volume_df + borrowing_volume_df).replace(0, np.nan)

In [12]:
df['max_test'] = df[['MAX Supply CMC', 'Total Supply CMC']].max(axis=1)
df['fully_diluted_mkt_cap'] = df.max_test * df.Price

del df['max_test']

In [13]:
df["total_revenue"] = df["Transaction Revenue"] + df["Interest Revenue"]
df["total_supply_revenue"] = df["Transaction Revenue Supply Side"] + df["Interest Revenue Supply Side"]
df["total_protocol_revenue"] = df["Interest Revenue Protocol"] + df["Transaction Revenue Protocol"]


In [14]:
df[["total_revenue", "total_supply_revenue", "total_protocol_revenue"]]

,total_revenue,total_supply_revenue,total_protocol_revenue
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
36302,NaN,NaN,NaN
36303,NaN,NaN,NaN
36304,NaN,NaN,NaN
36305,NaN,NaN,NaN


In [15]:
df = df.sort_values('date')

In [16]:
cmc_symbol_name_df = pd.read_sql_query("select distinct Symbol, Name from g_master_data_joins_2 order by Symbol, Name", __connect__())

In [17]:
cmc_symbol_name_df

,Symbol,Name
0,1INCH,1inch
1,AAVE,Aave
2,ABAT,Aave BAT
3,ABUSD,Aave BUSD
4,ACED,AceD
...,...,...
455,ZDEX,Zeedex
456,ZEC,Zcash
457,ZEE,ZeroSwap
458,ZKS,ZKSwap


In [18]:
def calculate_nDay_revenue(df, days):
    column_name = f"total_revenue_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['total_revenue'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [19]:
def calculate_nDay_trading_volume(df, days):
    column_name = f"total_volume_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['Volume CMC'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]

In [20]:
# function for defi volume sums
# substitute for volume cmc sum
def calculate_nDay_defi_volume(df, days):
    column_name = f"total_defi_volume_{days}d"
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df['Defi Volume DeBank'].rolling(f"{days}d").sum()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]

In [21]:
calculate_nDay_revenue(df, 7)
calculate_nDay_revenue(df, 30)
# calculate_nDay_trading_volume(df, 30)
calculate_nDay_defi_volume(df, 30)


In [22]:
df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank', 'TVL Defipulse', 'Transaction Revenue',
       'Interest Revenue', 'Transaction Revenue Supply Side',
       'Interest Revenue Supply Side', 'Transaction Revenue Protocol',
       'Interest Revenue Protocol', 'Transaction GMV', 'Interest GMV',
       'Price/Sale', 'Users 24hr Debank', 'Transactions Count Debank',
       'Trading Volume Debank', 'Borrowing Volume Debank',
       'Users life-to-date duneanlytics', 'CoinUrl', 'Tags', 'TVL_max', 'GMV',
       'Defi Volume DeBank', 'fully_diluted_mkt_cap', 'total_revenue',
       'total_supply_revenue', 'total_protocol_revenue', 'total_revenue_7d',
       'total_revenue_30d', 'total_defi_volume_30d'],
      dtype='object')

In [23]:
df[df['Symbol'] == 'AAVE']['total_revenue_7d']

65       5.105896e+04
293      1.039415e+05
360      1.514182e+05
658      2.232560e+05
728      2.762748e+05
             ...     
34447    2.116004e+06
34923    1.824975e+06
35086    1.795100e+06
35668    1.470444e+06
35970    1.088744e+06
Name: total_revenue_7d, Length: 109, dtype: float64

In [24]:
# df[df['Symbol'] == 'AAVE']['total_volume_30d']

In [25]:
df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,TVL_max,GMV,Defi Volume DeBank,fully_diluted_mkt_cap,total_revenue,total_supply_revenue,total_protocol_revenue,total_revenue_7d,total_revenue_30d,total_defi_volume_30d
0,2020-12-21,Amp,AMP,0.006596,6.361912e+07,3.698943e+05,9.645497e+09,1.200105e+09,9.239914e+10,NaN,...,NaN,NaN,NaN,6.094400e+08,NaN,NaN,NaN,NaN,NaN,NaN
115,2020-12-21,Standard Tokenization Protocol,STPT,0.019150,1.754746e+07,7.888539e+06,9.163266e+08,NaN,1.942420e+09,NaN,...,NaN,NaN,NaN,3.719694e+07,NaN,NaN,NaN,NaN,NaN,NaN
116,2020-12-21,SUKU,SUKU,0.136485,1.099190e+07,4.129467e+05,8.053551e+07,NaN,1.500000e+09,NaN,...,NaN,NaN,NaN,2.047276e+08,NaN,NaN,NaN,NaN,NaN,NaN
117,2020-12-21,inSure,SURE,0.001968,7.003992e+06,1.222256e+04,3.559703e+09,8.800000e+10,8.800000e+10,NaN,...,NaN,NaN,NaN,1.731468e+08,NaN,NaN,NaN,NaN,NaN,NaN
118,2020-12-21,SushiSwap,SUSHI,2.514379,4.298866e+08,1.834664e+08,1.709713e+08,0.000000e+00,1.709713e+08,1.114148e+09,...,1.164305e+09,9.021122e+07,8.481708e+07,4.298866e+08,NaN,NaN,NaN,NaN,NaN,8.481708e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36030,2021-04-08,DeFiner,FIN,0.737642,1.269010e+07,4.490922e+05,1.720360e+07,NaN,1.680000e+08,6.794976e+05,...,6.794976e+05,NaN,NaN,1.239239e+08,NaN,NaN,NaN,NaN,NaN,NaN
36029,2021-04-08,Radix,EXRD,0.143171,1.052070e+08,2.525191e+06,7.348346e+08,4.410000e+09,0.000000e+00,2.326637e+07,...,2.326637e+07,NaN,NaN,6.313842e+08,NaN,NaN,NaN,NaN,NaN,NaN
36028,2021-04-08,Ethereum Yield,ETHY,5.092072,5.092072e+05,2.452290e+03,1.000000e+05,5.000000e+05,5.000000e+05,NaN,...,NaN,NaN,NaN,2.546036e+06,NaN,NaN,NaN,NaN,NaN,NaN
36038,2021-04-08,Helmet.insure,HELMET,2.159960,5.762118e+07,6.021838e+06,2.667696e+07,1.000000e+08,1.000000e+08,NaN,...,NaN,NaN,NaN,2.159960e+08,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank', 'TVL Defipulse', 'Transaction Revenue',
       'Interest Revenue', 'Transaction Revenue Supply Side',
       'Interest Revenue Supply Side', 'Transaction Revenue Protocol',
       'Interest Revenue Protocol', 'Transaction GMV', 'Interest GMV',
       'Price/Sale', 'Users 24hr Debank', 'Transactions Count Debank',
       'Trading Volume Debank', 'Borrowing Volume Debank',
       'Users life-to-date duneanlytics', 'CoinUrl', 'Tags', 'TVL_max', 'GMV',
       'Defi Volume DeBank', 'fully_diluted_mkt_cap', 'total_revenue',
       'total_supply_revenue', 'total_protocol_revenue', 'total_revenue_7d',
       'total_revenue_30d', 'total_defi_volume_30d'],
      dtype='object')

In [27]:
# df['Marketcap CMC'] = df['Marketcap CMC'] + 0.01
# df['fully_diluted_mkt_cap'] = df['fully_diluted_mkt_cap'] + 0.01
# df['TVL Debank'] = df['TVL Debank'] + 0.01
# df['Users 24hr Debank'] = df['Users 24hr Debank'] + 0.01
# df['Transactions Count Debank'] = df['Transactions Count Debank'] + 0.01
# df['total_volume_30d'] = df['total_volume_30d'] + 0.01
# df['Trading Volume Debank'] = df['Trading Volume Debank'] + 0.01
# df['total_revenue'] = df['total_revenue'] + 0.01

In [28]:
# df['VOL : MC'] = df['Volume CMC'] / df['Marketcap CMC']
# df['VOL : DMC'] = df['Volume CMC'] / df['fully_diluted_mkt_cap']
# df['MC : TVL'] = df['Marketcap CMC'] / df['TVL_max']
# df['DMC : TVL'] = df['fully_diluted_mkt_cap'] / df['TVL_max']
# df['MC : users (1d)'] = df['Marketcap CMC'] / df['Users 24hr Debank']
# df['DMC : users (1d)'] = df['fully_diluted_mkt_cap'] / df['Users 24hr Debank']
# df['MC : transactions (1d)'] = df['Marketcap CMC'] / df['Transactions Count Debank']
# df['DMC : transactions (1d)'] = df['fully_diluted_mkt_cap'] / df['Transactions Count Debank']
# df['MC : DEX Vol (1d)'] = df['Marketcap CMC'] / df['Trading Volume Debank']
# df['MC : DEX Vol (30d)'] = df['Marketcap CMC'] / df['total_volume_30d']
# df['DMC : DEX Vol (1d)'] = df['fully_diluted_mkt_cap'] / df['Trading Volume Debank']
# df['DMC : DEX Vol (30d)'] = df['fully_diluted_mkt_cap'] / df['total_volume_30d']
# df['MC : revenue'] = df['Marketcap CMC'] / df['total_revenue']
# df['DMC : revenue'] = df['fully_diluted_mkt_cap'] / df['total_revenue']
# df['MC : policies sold'] = df['Marketcap CMC'] / df['abc']
# df['DMC : policies sold'] = df['fully_diluted_mkt_cap'] / df['abc']
# df['MC : premium'] = df['Marketcap CMC'] / df['abc']
# df['DMC : premium'] = df['fully_diluted_mkt_cap'] / df['abc']

In [29]:
df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,TVL_max,GMV,Defi Volume DeBank,fully_diluted_mkt_cap,total_revenue,total_supply_revenue,total_protocol_revenue,total_revenue_7d,total_revenue_30d,total_defi_volume_30d
0,2020-12-21,Amp,AMP,0.006596,6.361912e+07,3.698943e+05,9.645497e+09,1.200105e+09,9.239914e+10,NaN,...,NaN,NaN,NaN,6.094400e+08,NaN,NaN,NaN,NaN,NaN,NaN
115,2020-12-21,Standard Tokenization Protocol,STPT,0.019150,1.754746e+07,7.888539e+06,9.163266e+08,NaN,1.942420e+09,NaN,...,NaN,NaN,NaN,3.719694e+07,NaN,NaN,NaN,NaN,NaN,NaN
116,2020-12-21,SUKU,SUKU,0.136485,1.099190e+07,4.129467e+05,8.053551e+07,NaN,1.500000e+09,NaN,...,NaN,NaN,NaN,2.047276e+08,NaN,NaN,NaN,NaN,NaN,NaN
117,2020-12-21,inSure,SURE,0.001968,7.003992e+06,1.222256e+04,3.559703e+09,8.800000e+10,8.800000e+10,NaN,...,NaN,NaN,NaN,1.731468e+08,NaN,NaN,NaN,NaN,NaN,NaN
118,2020-12-21,SushiSwap,SUSHI,2.514379,4.298866e+08,1.834664e+08,1.709713e+08,0.000000e+00,1.709713e+08,1.114148e+09,...,1.164305e+09,9.021122e+07,8.481708e+07,4.298866e+08,NaN,NaN,NaN,NaN,NaN,8.481708e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36030,2021-04-08,DeFiner,FIN,0.737642,1.269010e+07,4.490922e+05,1.720360e+07,NaN,1.680000e+08,6.794976e+05,...,6.794976e+05,NaN,NaN,1.239239e+08,NaN,NaN,NaN,NaN,NaN,NaN
36029,2021-04-08,Radix,EXRD,0.143171,1.052070e+08,2.525191e+06,7.348346e+08,4.410000e+09,0.000000e+00,2.326637e+07,...,2.326637e+07,NaN,NaN,6.313842e+08,NaN,NaN,NaN,NaN,NaN,NaN
36028,2021-04-08,Ethereum Yield,ETHY,5.092072,5.092072e+05,2.452290e+03,1.000000e+05,5.000000e+05,5.000000e+05,NaN,...,NaN,NaN,NaN,2.546036e+06,NaN,NaN,NaN,NaN,NaN,NaN
36038,2021-04-08,Helmet.insure,HELMET,2.159960,5.762118e+07,6.021838e+06,2.667696e+07,1.000000e+08,1.000000e+08,NaN,...,NaN,NaN,NaN,2.159960e+08,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
def calculate_nDay_change(df, days, column, column_name):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].pct_change(days)
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

In [31]:
def calculate_nDay_std(df, days, column, column_name):
    df[column_name] = 0
    for index, row in cmc_symbol_name_df.iterrows():
        symbol = row['Symbol']
        name = row['Name']
        temp_df = df[(df['Symbol'] == symbol) & (df['Name'] == name)].copy()
        temp_df['parent_index'] = temp_df.index
        temp_df.index = pd.DatetimeIndex(temp_df['date'])
        temp_df[column_name] = temp_df[column].rolling(days).std()
        temp_df.index = temp_df["parent_index"]
        del temp_df['parent_index']
        for index, row in temp_df.iterrows():
            df.loc[index, column_name] = row[column_name]
    

## New Computations added by Time

In [32]:
# add 1, 3, 7, 30 day defi volume pct change here
# added column calculations
calculate_nDay_change(df, 1, 'Defi Volume DeBank', 'Defi_volume_pct_chg_1d')
calculate_nDay_change(df, 3, 'Defi Volume DeBank', 'Defi_volume_pct_chg_3d')
calculate_nDay_change(df, 7, 'Defi Volume DeBank', 'Defi_volume_pct_chg_7d')
calculate_nDay_change(df, 30, 'Defi Volume DeBank', 'Defi_volume_pct_chg_30d')



df['VOL : MC'] = df['Volume CMC'] / df['Marketcap CMC']
df['VOL : DMC'] = df['Volume CMC'] / df['fully_diluted_mkt_cap']

df['MC : TVL'] = df['Marketcap CMC'] / df['TVL_max']
# added column calculations
calculate_nDay_change(df, 1, 'MC : TVL', 'MC_TVL_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : TVL', 'MC_TVL_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : TVL', 'MC_TVL_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : TVL', 'MC_TVL_pct_chg_30d')
#


df['DMC : TVL'] = df['fully_diluted_mkt_cap'] / df['TVL_max']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : TVL', 'DMC_TVL_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : TVL', 'DMC_TVL_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : TVL', 'DMC_TVL_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : TVL', 'DMC_TVL_pct_chg_30d')
#


df['MC : users (1d)'] = df['Marketcap CMC'] / df['Users 24hr Debank']
# added column calculations
calculate_nDay_change(df, 1, 'MC : users (1d)', 'MC_users_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : users (1d)', 'MC_users_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : users (1d)', 'MC_users_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : users (1d)', 'MC_users_pct_chg_30d')
#




df['DMC : users (1d)'] = df['fully_diluted_mkt_cap'] / df['Users 24hr Debank']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : users (1d)', 'DMC_users_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : users (1d)', 'DMC_users_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : users (1d)', 'DMC_users_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : users (1d)', 'DMC_users_pct_chg_30d')
#



df['MC : transactions (1d)'] = df['Marketcap CMC'] / df['Transactions Count Debank']
# added column calculations
calculate_nDay_change(df, 1, 'MC : transactions (1d)', 'MC_transactions_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : transactions (1d)', 'MC_transactions_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : transactions (1d)', 'MC_transactions_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : transactions (1d)', 'MC_transactions_pct_chg_30d')
#

df['DMC : transactions (1d)'] = df['fully_diluted_mkt_cap'] / df['Transactions Count Debank']
# added column calculations
calculate_nDay_change(df, 1, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_1d')
calculate_nDay_change(df, 3, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_3d')
calculate_nDay_change(df, 7, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_7d')
calculate_nDay_change(df, 30, 'DMC : transactions (1d)', 'DMC_transactions_pct_chg_30d')
#

#df['MC : DEX Vol (1d)'] = df['Marketcap CMC'] / df['Trading Volume Debank']
# add
df['MC : Defi Vol (1d)'] = df['Marketcap CMC'] / df['Defi Volume DeBank']
# added column calculations
calculate_nDay_change(df, 1, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_1d')
calculate_nDay_change(df, 3, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_3d')
calculate_nDay_change(df, 7, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_7d')
calculate_nDay_change(df, 30, 'MC : Defi Vol (1d)', 'MC_defi_volume_pct_chg_30d')
#


df['MC : Defi Vol (30d)'] = df['Marketcap CMC'] / df['total_defi_volume_30d']
#df['DMC : DEX Vol (1d)'] = df['fully_diluted_mkt_cap'] / df['Trading Volume Debank']
#df['DMC : DEX Vol (30d)'] = df['fully_diluted_mkt_cap'] / df['total_volume_30d']



df['MC : revenue'] = df['Marketcap CMC'] / df['total_revenue']
df['DMC : revenue'] = df['fully_diluted_mkt_cap'] / df['total_revenue']
# df['MC : policies sold'] = df['Marketcap CMC'] / df['abc']
# df['DMC : policies sold'] = df['fully_diluted_mkt_cap'] / df['abc']
# df['MC : premium'] = df['Marketcap CMC'] / df['abc']
# df['DMC : premium'] = df['fully_diluted_mkt_cap'] / df['abc']

In [33]:
calculate_nDay_change(df, 1, 'Price', 'Price_pct_chg_1d')
calculate_nDay_change(df, 3, 'Price', 'Price_pct_chg_3d')
calculate_nDay_change(df, 7, 'Price', 'Price_pct_chg_7d')
calculate_nDay_change(df, 30, 'Price', 'Price_pct_chg_30d')
calculate_nDay_std(df, 7, 'Price', 'Price_7day_volatility_pct')
calculate_nDay_std(df, 30, 'Price', 'Price_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Price', 'Price_90day_volatility_pct')

In [34]:
calculate_nDay_change(df, 1, 'Marketcap CMC', 'MC_pct_chg_1d')
calculate_nDay_change(df, 3, 'Marketcap CMC', 'MC_pct_chg_3d')
calculate_nDay_change(df, 7, 'Marketcap CMC', 'MC_pct_chg_7d')
calculate_nDay_change(df, 30, 'Marketcap CMC', 'MC_pct_chg_30d')
calculate_nDay_std(df, 30, 'Marketcap CMC', 'MC_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Marketcap CMC', 'MC_90day_volatility_pct')

In [35]:
calculate_nDay_change(df, 1, 'fully_diluted_mkt_cap', 'DMC_pct_chg_1d')
calculate_nDay_change(df, 3, 'fully_diluted_mkt_cap', 'DMC_pct_chg_3d')
calculate_nDay_change(df, 7, 'fully_diluted_mkt_cap', 'DMC_pct_chg_7d')
calculate_nDay_change(df, 30, 'fully_diluted_mkt_cap', 'DMC_pct_chg_30d')
calculate_nDay_std(df, 30, 'fully_diluted_mkt_cap', 'DMC_30day_volatility_pct')
calculate_nDay_std(df, 90, 'fully_diluted_mkt_cap', 'DMC_90day_volatility_pct')

In [36]:
calculate_nDay_change(df, 1, 'Circulating supply CMC', 'CS_pct_chg_1d')
calculate_nDay_change(df, 3, 'Circulating supply CMC', 'CS_pct_chg_d')
calculate_nDay_change(df, 7, 'Circulating supply CMC', 'CS_pct_chg_7d')
calculate_nDay_change(df, 30, 'Circulating supply CMC', 'CS_pct_chg_30d')
calculate_nDay_std(df, 30, 'Circulating supply CMC', 'CS_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Circulating supply CMC', 'CS_90day_volatility_pct')

In [37]:
calculate_nDay_change(df, 1, 'Volume CMC', 'Volume_pct_chg_1d')
calculate_nDay_change(df, 3, 'Volume CMC', 'Volume_pct_chg_3d')
calculate_nDay_change(df, 7, 'Volume CMC', 'Volume_pct_chg_7d')
calculate_nDay_change(df, 30, 'Volume CMC', 'Volume_pct_chg_30d')
calculate_nDay_std(df, 30, 'Volume CMC', 'Volume_30day_volatility_pct')
calculate_nDay_std(df, 90, 'Volume CMC', 'Volume_90day_volatility_pct')

In [38]:
calculate_nDay_change(df, 1, 'TVL_max', 'TVL_pct_chg_1d')
calculate_nDay_change(df, 3, 'TVL_max', 'TVL_pct_chg_3d')
calculate_nDay_change(df, 7, 'TVL_max', 'TVL_pct_chg_7d')
calculate_nDay_change(df, 30, 'TVL_max', 'TVL_pct_chg_30d')
calculate_nDay_std(df, 30, 'TVL_max', 'TVL_30day_volatility_pct')
calculate_nDay_std(df, 90, 'TVL_max', 'TVL_90day_volatility_pct')

In [39]:
calculate_nDay_change(df, 1, 'total_revenue', 'Rev_pct_chg_1d')
calculate_nDay_change(df, 3, 'total_revenue', 'Rev_pct_chg_3d')
calculate_nDay_change(df, 7, 'total_revenue', 'Rev_pct_chg_7d')
calculate_nDay_change(df, 30, 'total_revenue', 'Rev_pct_chg_30d')
calculate_nDay_std(df, 30, 'total_revenue', 'Rev_30day_volatility_pct')
calculate_nDay_std(df, 90, 'total_revenue', 'Rev_90day_volatility_pct')

In [40]:
# calculate_nDay_change(df, 1, 'MC : TVL', 'MC : TVL_pct_chg_1d')
# calculate_nDay_change(df, 3, 'MC : TVL', 'MC : TVL_pct_chg_3d')
# calculate_nDay_change(df, 7, 'MC : TVL', 'MC : TVL_pct_chg_7d')
# calculate_nDay_change(df, 30, 'MC : TVL', 'MC : TVL_pct_chg_30d')
# calculate_nDay_std(df, 30, 'Circulating supply CMC', 'Rev σ 30d')
# calculate_nDay_std(df, 90, 'Circulating supply CMC', 'Rev σ 90d')

In [41]:
# calculate_nDay_change(df, 1, 'DMC : TVL', 'DMC : TVL_pct_chg_1d')
# calculate_nDay_change(df, 3, 'DMC : TVL', 'DMC : TVL_pct_chg_3d')
# calculate_nDay_change(df, 7, 'DMC : TVL', 'DMC : TVL_pct_chg_7d')
# calculate_nDay_change(df, 30, 'DMC : TVL', 'DMC : TVL_pct_chg_30d')
# calculate_nDay_std(df, 30, 'Circulating supply CMC', 'CS σ 30d')
# calculate_nDay_std(df, 90, 'Circulating supply CMC', 'CS σ 90d')

In [42]:
df['Annual revenue (estimate)'] = df['total_revenue_30d'] * (1 + df['Rev_pct_chg_30d'])*12

In [43]:
df['Annual revenue (estimate)'].unique()

array([           nan, 2.02762697e+08, 1.79198565e+08, 1.55136950e+08,
       9.82067771e+07, 1.40273154e+08, 1.73302609e+08, 1.18109496e+08,
       1.30405992e+08, 7.82277420e+07, 1.06151542e+08, 4.83362838e+07,
       3.54432943e+07, 3.57654763e+07, 2.42611508e+07, 3.62822740e+07,
       9.78291443e+08, 1.58607172e+08, 1.24916389e+08, 1.08150777e+08,
       8.31272407e+07, 1.67808110e+08, 1.82386531e+08, 1.12688990e+08,
       1.85816255e+08, 2.53641482e+08, 1.99097480e+08, 1.94388793e+08,
       2.48472942e+08, 2.68241292e+08, 2.29419238e+08, 2.08560426e+08,
       1.82448061e+08, 2.69880318e+08, 3.28002294e+08, 2.74389703e+08,
       1.67887901e+08, 1.79530880e+08, 2.21814828e+08, 1.98902733e+08,
       2.11631211e+08, 1.79019530e+08, 2.21742618e+08, 1.75270421e+08,
       2.43850185e+08, 1.51511963e+08, 1.32118698e+07, 1.09762841e+08,
       2.81240136e+08, 3.09942448e+08, 3.66379107e+08, 2.37816658e+08,
       1.61882767e+08, 1.22663961e+08, 1.10058266e+08, 1.03998132e+08,
      

In [44]:
df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank',
       ...
       'TVL_pct_chg_30d', 'TVL_30day_volatility_pct',
       'TVL_90day_volatility_pct', 'Rev_pct_chg_1d', 'Rev_pct_chg_3d',
       'Rev_pct_chg_7d', 'Rev_pct_chg_30d', 'Rev_30day_volatility_pct',
       'Rev_90day_volatility_pct', 'Annual revenue (estimate)'],
      dtype='object', length=125)

In [45]:
df[(df['Symbol'] == 'ZRX')]['GMV']

179      6.533663e+07
234      5.306118e+07
543      6.193404e+07
598      6.455556e+07
912      5.017659e+07
             ...     
34678    1.164772e+08
34748             NaN
35316    1.522847e+08
35899             NaN
35969             NaN
Name: GMV, Length: 109, dtype: float64

In [46]:
df.to_csv('master_data_ratios_and_percentages.csv')

In [47]:
df.shape

(36307, 125)

In [48]:
required_tags = ['binance-smart-chain','polkadot-ecosystem','ethereum','cosmos-ecosystem','polkadot','solana-ecosystem','binance-chain','avalanche-ecosystem','heco','layer-2','eos','sidechain','cosmos','tron','defi','yield-farming','decentralized-exchange','governance','dao','gaming','stablecoin','asset-management','centralized-exchange','staking','yield-aggregator','oracles','lending-borowing','dex','options','launchpad','stablecoin-algorithmically-stabilized']
required_tags.sort()
required_tags

['asset-management',
 'avalanche-ecosystem',
 'binance-chain',
 'binance-smart-chain',
 'centralized-exchange',
 'cosmos',
 'cosmos-ecosystem',
 'dao',
 'decentralized-exchange',
 'defi',
 'dex',
 'eos',
 'ethereum',
 'gaming',
 'governance',
 'heco',
 'launchpad',
 'layer-2',
 'lending-borowing',
 'options',
 'oracles',
 'polkadot',
 'polkadot-ecosystem',
 'sidechain',
 'solana-ecosystem',
 'stablecoin',
 'stablecoin-algorithmically-stabilized',
 'staking',
 'tron',
 'yield-aggregator',
 'yield-farming']

In [49]:
data_df = df
data_df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,TVL_pct_chg_30d,TVL_30day_volatility_pct,TVL_90day_volatility_pct,Rev_pct_chg_1d,Rev_pct_chg_3d,Rev_pct_chg_7d,Rev_pct_chg_30d,Rev_30day_volatility_pct,Rev_90day_volatility_pct,Annual revenue (estimate)
0,2020-12-21,Amp,AMP,0.006596,6.361912e+07,3.698943e+05,9.645497e+09,1.200105e+09,9.239914e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,2020-12-21,Standard Tokenization Protocol,STPT,0.019150,1.754746e+07,7.888539e+06,9.163266e+08,NaN,1.942420e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,2020-12-21,SUKU,SUKU,0.136485,1.099190e+07,4.129467e+05,8.053551e+07,NaN,1.500000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2020-12-21,inSure,SURE,0.001968,7.003992e+06,1.222256e+04,3.559703e+09,8.800000e+10,8.800000e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,2020-12-21,SushiSwap,SUSHI,2.514379,4.298866e+08,1.834664e+08,1.709713e+08,0.000000e+00,1.709713e+08,1.114148e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36030,2021-04-08,DeFiner,FIN,0.737642,1.269010e+07,4.490922e+05,1.720360e+07,NaN,1.680000e+08,6.794976e+05,...,-0.270797,90506.895747,3.183276e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36029,2021-04-08,Radix,EXRD,0.143171,1.052070e+08,2.525191e+06,7.348346e+08,4.410000e+09,0.000000e+00,2.326637e+07,...,-0.054734,992833.405650,3.240611e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36028,2021-04-08,Ethereum Yield,ETHY,5.092072,5.092072e+05,2.452290e+03,1.000000e+05,5.000000e+05,5.000000e+05,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36038,2021-04-08,Helmet.insure,HELMET,2.159960,5.762118e+07,6.021838e+06,2.667696e+07,1.000000e+08,1.000000e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
tag_df = pd.DataFrame(data_df['Tags'].copy())
tag_df

,Tags
0,
115,"defi,derivatives,payments,polkadot-ecosystem,s..."
116,defi
117,"defi,insurance,polkadot-ecosystem"
118,"decentralized-exchange,defi,memes,dao,yield-fa..."
...,...
36030,"defi,lending-borowing,dao-maker"
36029,ethereum
36028,"defi,binance-smart-chain"
36038,"insurance,binance-smart-chain"


In [51]:
required_tags_map = {}
i = 0
for tag in required_tags:
    required_tags_map[tag] = i
    i = i + 1

In [52]:
required_tags_map

{'asset-management': 0,
 'avalanche-ecosystem': 1,
 'binance-chain': 2,
 'binance-smart-chain': 3,
 'centralized-exchange': 4,
 'cosmos': 5,
 'cosmos-ecosystem': 6,
 'dao': 7,
 'decentralized-exchange': 8,
 'defi': 9,
 'dex': 10,
 'eos': 11,
 'ethereum': 12,
 'gaming': 13,
 'governance': 14,
 'heco': 15,
 'launchpad': 16,
 'layer-2': 17,
 'lending-borowing': 18,
 'options': 19,
 'oracles': 20,
 'polkadot': 21,
 'polkadot-ecosystem': 22,
 'sidechain': 23,
 'solana-ecosystem': 24,
 'stablecoin': 25,
 'stablecoin-algorithmically-stabilized': 26,
 'staking': 27,
 'tron': 28,
 'yield-aggregator': 29,
 'yield-farming': 30}

In [53]:
all_symbol_tags = []
for index, row in tag_df.iterrows():
    tag = [''] * (len(required_tags))
    tags = row['Tags']
    tags = tags.split(',') if tags else []
    for t in tags:
        index = required_tags_map[t] if t in required_tags_map else -1
        if index > -1:
            tag[index] = 1
    all_symbol_tags.append(tag)
        

In [54]:
all_symbol_tags

[['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  1,
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  1,
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  1,
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  1,
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  1,
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  1,
  1,
  1,
  '',
  '',
  '',
  '',
  1,
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  1],
 ['',
  '',
  '',
  1,
  '',
  '',
  '',
  '',
  '',
  1,
  '',
  '',
  '',

In [55]:
columns = []
columns.extend(required_tags)
columns

['asset-management',
 'avalanche-ecosystem',
 'binance-chain',
 'binance-smart-chain',
 'centralized-exchange',
 'cosmos',
 'cosmos-ecosystem',
 'dao',
 'decentralized-exchange',
 'defi',
 'dex',
 'eos',
 'ethereum',
 'gaming',
 'governance',
 'heco',
 'launchpad',
 'layer-2',
 'lending-borowing',
 'options',
 'oracles',
 'polkadot',
 'polkadot-ecosystem',
 'sidechain',
 'solana-ecosystem',
 'stablecoin',
 'stablecoin-algorithmically-stabilized',
 'staking',
 'tron',
 'yield-aggregator',
 'yield-farming']

In [56]:
df2 = pd.DataFrame(all_symbol_tags, columns = columns)
df2

,asset-management,avalanche-ecosystem,binance-chain,binance-smart-chain,centralized-exchange,cosmos,cosmos-ecosystem,dao,decentralized-exchange,defi,...,polkadot,polkadot-ecosystem,sidechain,solana-ecosystem,stablecoin,stablecoin-algorithmically-stabilized,staking,tron,yield-aggregator,yield-farming
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,1,...,,1,,,,,,,,
2,,,,,,,,,,1,...,,,,,,,,,,
3,,,,,,,,,,1,...,,1,,,,,,,,
4,,,,,,,,1,1,1,...,,,,,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36302,,,,,,,,,,1,...,,,,,,,,,,
36303,,,,,,,,,,,...,,,,,,,,,,
36304,,,,1,,,,,,1,...,,,,,,,,,,
36305,,,,1,,,,,,,...,,,,,,,,,,


In [57]:
data_df = data_df.join(df2)
del data_df['Tags']
del data_df['CoinUrl']

In [58]:
data_df

,date,Name,Symbol,Price,Marketcap CMC,Volume CMC,Circulating supply CMC,MAX Supply CMC,Total Supply CMC,TVL Debank,...,polkadot,polkadot-ecosystem,sidechain,solana-ecosystem,stablecoin,stablecoin-algorithmically-stabilized,staking,tron,yield-aggregator,yield-farming
0,2020-12-21,Amp,AMP,0.006596,6.361912e+07,3.698943e+05,9.645497e+09,1.200105e+09,9.239914e+10,NaN,...,,,,,,,,,,
115,2020-12-21,Standard Tokenization Protocol,STPT,0.019150,1.754746e+07,7.888539e+06,9.163266e+08,NaN,1.942420e+09,NaN,...,,,,,,,,,,1
116,2020-12-21,SUKU,SUKU,0.136485,1.099190e+07,4.129467e+05,8.053551e+07,NaN,1.500000e+09,NaN,...,,,,,,,,,,
117,2020-12-21,inSure,SURE,0.001968,7.003992e+06,1.222256e+04,3.559703e+09,8.800000e+10,8.800000e+10,NaN,...,,,,,,,,,1,1
118,2020-12-21,SushiSwap,SUSHI,2.514379,4.298866e+08,1.834664e+08,1.709713e+08,0.000000e+00,1.709713e+08,1.114148e+09,...,,,,1,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36030,2021-04-08,DeFiner,FIN,0.737642,1.269010e+07,4.490922e+05,1.720360e+07,NaN,1.680000e+08,6.794976e+05,...,,,,,,,,,,
36029,2021-04-08,Radix,EXRD,0.143171,1.052070e+08,2.525191e+06,7.348346e+08,4.410000e+09,0.000000e+00,2.326637e+07,...,,,,,,,,,,
36028,2021-04-08,Ethereum Yield,ETHY,5.092072,5.092072e+05,2.452290e+03,1.000000e+05,5.000000e+05,5.000000e+05,NaN,...,,,,,,,,,,
36038,2021-04-08,Helmet.insure,HELMET,2.159960,5.762118e+07,6.021838e+06,2.667696e+07,1.000000e+08,1.000000e+08,NaN,...,,,,,,,,,,


In [59]:
data_df.to_csv('master_data_ratios_and_percentages.csv')

In [60]:
data_df.shape

(36307, 154)

In [61]:
data_df.columns

Index(['date', 'Name', 'Symbol', 'Price', 'Marketcap CMC', 'Volume CMC',
       'Circulating supply CMC', 'MAX Supply CMC', 'Total Supply CMC',
       'TVL Debank',
       ...
       'polkadot', 'polkadot-ecosystem', 'sidechain', 'solana-ecosystem',
       'stablecoin', 'stablecoin-algorithmically-stabilized', 'staking',
       'tron', 'yield-aggregator', 'yield-farming'],
      dtype='object', length=154)